# Gestire dati mancanti
L'operare su dataset in cui alcuni valori sono mancanti è un problema tipico del data preprocessing.
Vediamo i metodi da applicare in questo caso, cominciamo importando le librerie che utilizzeremo.

In [3]:
import pandas as pd
import numpy as np

## Creiamo il nostro dataset con valori mancanti
Per i nostri esempi utilizzeremo l'Iris Dataset, questo famoso dataset non presenta alcun valore mancante, quindi creiamone qualcuno noi.<br>
Carichiamo il dataset in un DataFrame.

In [4]:
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", 
                   names=["sepal_length","sepal_width","petal_length","petal_width","class"])
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
iris_nan = iris.copy()
max_val = iris.shape[0]

samples = np.random.randint(max_val, size=(10)) #Creiamo un vettore di 10 numeri casuali tra 0 e il numero di osservazioni
iris_nan.loc[samples,'petal_length']=None #Sostituiamo il valore di "petal_length" per ognuna delle 10 osservazioni con un valore non valido

nan_count = iris_nan['petal_length'].isnull().sum() #contiamo il numero di valori non validi all'interno della colonna "petal_legnth"
print("Il dataset ha "+str(nan_count)+" valori mancanti")
iris_nan.to_csv("data/iris_with_nan.csv") # salviamo il dataset così creato all'interno di un file CSV

Il dataset ha 10 valori mancanti


Utilizziamo il DataFrame per caricare il dataset anche in un array numpy

In [9]:
Y = iris_nan["class"].values
X = iris_nan.drop("class",axis=1).values

## - Metodo 1: Rimuovere proprietà o esempi con valori mancanti

Una soluzione drastica consiste nel rimuove gli esempi che presentano valori mancanti utilizzando il metodo dropna.

In [10]:
samples_before = iris_nan.shape[0]
iris_drop = iris_nan.dropna()

samples_after = iris_drop.shape[0]

print("Numero di esempi prima: "+str(samples_before))
print("Numero di esempi dopo: "+str(samples_after))

Numero di esempi prima: 150
Numero di esempi dopo: 140


Se i valori mancanti corrispondono ad un unica feature e questi sono in un numero tale da invalidare l'utilità della feature, allora possiamo semplicemente rimuovere la feature dal nostro DataFrame.

In [12]:
iris_cleaned = iris_nan.dropna(axis=1)
iris_cleaned.columns

Index(['sepal_length', 'sepal_width', 'petal_width', 'class'], dtype='object')

Rinunciare a dati preziosi non è mai una buona cosa, quindi questi metodi vanno evitati ad eccezione di casi estremi, ovvero quando la maggior parte dei valori per una feature o per un esempio sono mancanti.

## - Metodo 2: Imputazione dei dati mancanti

L'imputazione dei dati mancanti consiste nel sostituire i valori con una stima.<br>
Il metodo più comune è **l'imputazione con media** (mean imputation) in cui i valori mancanti vengono sostituiti con il valore medio della proprietà, altri metodi sono l'imputazione con la mediana o con valore più frequente (moda).

### Pandas
Con Pandas possiamo utilizzare il metodo fillna per sostituire i valori mancanti con le stime.

In [67]:
replace_with = iris_nan['petal_length'].mean() # imputazione con media
#replace_with = iris_nan['petal_length'].median() # imputazione con mediana
#replace_with = iris_nan['petal_length'].mode() # imputazione con moda
iris_nan['petal_length'].fillna(replace_with,inplace=True)
nan_count = iris_nan['petal_length'].isnull().sum() #verifichiamo che la colonna "petal_length" non contenga più valori non validi.
print("Il dataset ha "+str(nan_count)+" valori mancanti")

Il dataset ha 0 valori mancanti


### Numpy e Scikit-learn
Per eseguire l'imputazione di un array numpy possiamo utilizzare la classe Imputer di scikit-learn, il tipo di imputazione può essere specificata nella strategia (mean, median, most_frequent)

In [66]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values="NaN", strategy="mean", axis=0) 
X_imp = imp.fit_transform(X)
nan_count = np.count_nonzero(np.isnan(X_imp))
print("Il dataset ha "+str(nan_count)+" valori mancanti")

Il dataset ha 0 valori mancanti
